# ハルシネーションとRLHF

ハルシネーションとRLHFでは、モデル呼び出し以前に入力設計と評価設計を固める習慣を作ります。


## 概念の土台

ハルシネーションとRLHFに入る前に、つまずきやすい用語を先にそろえます。以降のコードでは、変数がどの概念を表しているかを対応付けながら読んでください。

- **トークン**: モデルが処理する最小単位です。日本語では1文字単位とは限りません。
- **次トークン確率**: 文脈に対して次に出る候補の確率分布です。生成の中核です。
- **コンテキスト**: モデルに渡す入力全体です。指示・資料・履歴が含まれます。
- **ハルシネーション**: 根拠がない内容をもっともらしく生成する現象です。
- **RLHF**: 人間の選好を報酬にして応答方針を調整する手法です。

このノートでは、ここで定義した語を実験セルの変数・式に直接対応させて確認します。


## 実験 1: ハルシネーション観測の基礎

根拠欠落の回答を減らす観点で、入力長と情報密度をまず確認します。


In [ ]:
text = '根拠が不足した回答は一見自然でも信頼性を損なう。'
char_len = len(text)
space_tokens = text.split()
rough_tokens = max(1, char_len // 2)
print('task = hallucination-rlhf')
print('chars=', char_len, 'space_tokens=', len(space_tokens), 'rough_tokens=', rough_tokens)

以降で評価軸と報酬設計に接続します。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 2: プロンプトを構造化する

次に、指示・制約・出力形式を分離したテンプレートを作ります。曖昧さを減らすための実装技法です。


In [ ]:
instruction = '勾配降下法を初学者向けに説明する'
constraints = ['120字以内', '比喩は1つまで', '最後に要点を1行でまとめる']
prompt = f"指示: {instruction}\n制約: {'; '.join(constraints)}\n出力:"
print(prompt)
print('prompt_chars=', len(prompt))

この形にすると、失敗原因を特定しやすくなります。品質改善は原因分離のしやすさで決まります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 式と実装の往復

1. $p_{\theta}(x_t \mid x_{<t})$
2. $L_{CE} = -\sum_t \log p_{\theta}(x_t \mid x_{<t})$


## 実験 3: 検索文脈を結合する

ここで RAG の最小形を実装します。質問と関連文を結合し、回答入力を作る流れを確認します。


In [ ]:
question = 'ベルマン方程式を高校生向けに説明して'
retrieved = ['価値は将来報酬の割引和で定義する', '現在価値は次状態価値で再帰的に更新できる']
context = '\n'.join(f'- {c}' for c in retrieved)
final_input = f"質問:\n{question}\n\n参考文脈:\n{context}\n\n回答:"
print(final_input)

検索文脈を入れる目的は、モデルの記憶に頼りすぎないことです。根拠付き応答を作りやすくなります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 4: 評価項目を数値化する

次に、回答を点検するための簡易スコアを定義します。評価軸を言語化すると改善が継続できます。


In [ ]:
answer = 'ベルマン方程式は、今の価値を次の価値で更新する再帰式です。'
checks = {'length_ok': len(answer) <= 120, 'has_keyword': '価値' in answer, 'has_recurrence': '再帰' in answer}
score = sum(1 for v in checks.values() if v) / len(checks)
print('checks=', checks)
print('score=', round(score, 3))

このような軽量評価でも、改善方向を揃える効果があります。実務ではこの評価軸をチームで共有します。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 実験 5: 推論コストを見積もる

最後に、入力と出力の長さから概算コストを計算します。モデル選択は性能だけでなく費用とのバランスが必要です。


In [ ]:
input_tokens = 320
output_tokens = 180
price_per_1k = 0.0012
cost = (input_tokens + output_tokens) / 1000 * price_per_1k
print('estimated_cost=', round(cost, 6))

この見積もりを運用前に作ると、スケール時の予算超過を防ぎやすくなります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 振り返り

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. プロンプトだけで全問題を解決しようとする
2. 評価指標を決めずに改善を繰り返す
3. コストと品質のバランスを見ない
